# Victim Analysis Notebook (SOC/IDS)

This notebook analyzes the `network_logs.csv` file generated by the attacker simulation.
It acts as a simple Intrusion Detection System (IDS) and Security Operations Center (SOC) dashboard.

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Load the logs
try:
    df = pd.read_csv('network_logs.csv')
    print("Logs loaded successfully.")
    print(f"Total entries: {len(df)}")
except FileNotFoundError:
    print("Error: network_logs.csv not found. Please run the attacker simulation first.")

In [3]:
# Display first few rows
df.head()

## 1. Traffic Overview

In [4]:
if 'df' in locals():
    plt.figure(figsize=(10, 6))
    sns.countplot(x='action', data=df, palette='viridis')
    plt.title('Distribution of Network Actions')
    plt.xticks(rotation=45)
    plt.show()

## 2. Intrusion Detection System (IDS) Logic

In [5]:
def detect_threats(df):
    threats = []
    
    # 1. DoS Detection: High volume of requests from single IP
    request_counts = df[df['action'] == 'HTTP_REQUEST']['source_ip'].value_counts()
    for ip, count in request_counts.items():
        if count > 100:  # Threshold
            threats.append({'IP': ip, 'Type': 'DoS Attack', 'Details': f'{count} requests detected'})

    # 2. Port Scan / Recon Detection: 
    # Check for Scanner User-Agents in details OR high port diversity
    recon_df = df[df['details'].str.contains('Scanner|Nmap', case=False, na=False)]
    for ip in recon_df['source_ip'].unique():
        threats.append({'IP': ip, 'Type': 'Network Recon', 'Details': 'Scanner Fingerprint Detected'})

    # 3. Brute Force Detection: High number of failed logins
    login_df = df[(df['action'] == 'LOGIN_ATTEMPT') & (df['details'].str.contains('Failed', na=False))]
    login_counts = login_df['source_ip'].value_counts()
    for ip, count in login_counts.items():
        if count > 5: # Threshold
            threats.append({'IP': ip, 'Type': 'Brute Force', 'Details': f'{count} failed attempts'})

    # 4. SQL Injection (Signature)
    sqli_df = df[(df['action'] == 'LOGIN_ATTEMPT') & 
                 (df['details'].str.contains("' OR|UNION SELECT|1=1", case=False, na=False))]
    for ip in sqli_df['source_ip'].unique():
         threats.append({'IP': ip, 'Type': 'SQL Injection', 'Details': 'SQLi payload detected'})

    # 5. XSS (Signature)
    xss_df = df[(df['action'] == 'SEARCH_QUERY') & 
                (df['details'].str.contains("<script>|alert(", case=False, na=False))]
    for ip in xss_df['source_ip'].unique():
         threats.append({'IP': ip, 'Type': 'XSS Attack', 'Details': 'XSS payload detected'})

    # 6. Command Injection (Signature) in Search
    ci_df = df[(df['action'] == 'SEARCH_QUERY') & 
               (df['details'].str.contains(";|/etc/passwd", case=False, na=False))]
    for ip in ci_df['source_ip'].unique():
         threats.append({'IP': ip, 'Type': 'Command Injection', 'Details': 'RCE payload detected'})
            
    return pd.DataFrame(threats)

if 'df' in locals():
    threats_df = detect_threats(df)
    print("Detected Threats:")
    print(threats_df)

## 3. Threat Visualization

In [6]:
if 'threats_df' in locals() and not threats_df.empty:
    plt.figure(figsize=(10, 6))
    sns.countplot(x='Type', data=threats_df, palette='Reds')
    plt.title('Detected Security Incidents')
    plt.show()
else:
    print("No threats detected (or logs not loaded).")